In [1]:
import numpy as np
import sympy as sp
from scipy import linalg

In [2]:
#Properties (N and mm)
E1 = 60800
E2 = 58250
v12 = 0.07
G12 = 4550
t = 0.21
v21 = v12 * E2 / E1

# Plate Parameters
a = 360
b = 360
m = 4
n = 4

#Boundary conditions
t1 = 0 # 0=> no translation, 1 => translation
t2 = 0 
r1 = 0 # 0 => there is rotation, 1 => null rotatiion 
r2 = 0

# Loads
Nxx = -1
Nyy = 0
Nxy = 0

**[Q] in the principal direction**  pg. 52 chap 2 of the three regions

In [3]:
Q_0 = np.array([[E1/(1-v12*v21), v12*E2/(1-v12*v21), 0],
                 [v12*E2/(1-v12*v21), E2/(1-v12*v21),0],
                 [0, 0, G12]])

**[Q] rotated in theta direction** pg. 53 and 54 of chap 2

In [4]:
layup = [(45,'f'),(0,'f'),(0,'f'),(45,'f'),(0,'f'),(0,'f'),(45,'f')]
Q_layup = []

#Layup assembly
for theta in layup:
    c = np.cos(theta[0]*np.pi/180)
    s = np.sin(theta[0]*np.pi/180)

    T_real = np.array([[c**2, s**2, 2*c*s],
                       [s**2, c**2, -2*c*s],
                       [-c*s, c*s, c**2-s**2]])

    T_engineering =  np.array([[c**2, s**2, c*s],
                               [s**2, c**2, -c*s],
                               [-2*c*s, 2*c*s, c**2-s**2]])


    if theta[1] == 'f':
        Q_layup.append((np.linalg.inv(T_real))@Q_0@T_engineering)


**[A], [B], [D]** pg. 24 fo chap 4 of the three regions

In [5]:
t_position = [-3.5*t, -2.5*t, -1.5*t, -0.5*t, 0.5*t, 1.5*t, 2.5*t, 3.5*t]

A = np.zeros(9).reshape(3,3)
B = np.zeros(9).reshape(3,3)
D = np.zeros(9).reshape(3,3)

for i in enumerate(Q_layup):
    zk1 = t_position[i[0]+1]
    zk0 = t_position[i[0]]
    
    A += (zk1 - zk0)*i[1]
    B += (1/2)*(zk1**2 - zk0**2)*i[1]
    D += (1/3)*(zk1**3 - zk0**3)*i[1]


F = np.vstack([
    np.hstack([A, B]),
    np.hstack([B, D])
])

F = sp.Matrix(F)

### Rayleigh-Ritz formulation

In [6]:
Ny = sp.symbols('Ny')
x, y = sp.symbols(['x', 'y'])

In [7]:
S = [[]]

for i in range(m):
    for j in range(n):
        S[0].append(sp.sin((i+1)*sp.pi*x/a)*sp.sin((j+1)*sp.pi*y/b))
S = sp.Matrix(S)


B0_11 = np.array(sp.diff(S, x))
B0_22 = np.array(sp.diff(S, y))
B0_31 = np.array(sp.diff(S, y))
B0_32 = np.array(sp.diff(S, x))
B0_43 = np.array(-1 * sp.diff(S, x, x))
B0_53 = np.array(-1 * sp.diff(S, y, y))
B0_63 = np.array(-2 * sp.diff(S, x, y))
Z = np.zeros(m*n).reshape(1, m*n)

B0_kappa = sp.Matrix([
    np.hstack([*B0_11, *Z, *Z]),
    np.hstack([*Z, *B0_22, *Z]),
    np.hstack([*B0_31, *B0_32, *Z]),
    np.hstack([*Z, *Z, *B0_43]),
    np.hstack([*Z, *Z, *B0_53]),
    np.hstack([*Z, *Z, *B0_63])
])


F = sp.Matrix(F)


G = sp.Matrix([
    np.hstack([*Z, *Z, *sp.diff(S, x)]),
    np.hstack([*Z, *Z, *sp.diff(S, y)])
 ])
G = sp.Matrix(G)

N = sp.Matrix([[Nxx, Nxy],
                [Nxy, Nyy]])

In [8]:
G.shape

(2, 48)

In [9]:
KG = sp.integrate(
     sp.integrate(
        -1 * G.T * N * G, (x, 0, a)), (y, 0, b)
)

KG = np.array(KG, dtype=np.longdouble)

In [10]:
print(KG.shape)
print(KG.min())
print(KG.max())

(48, 48)
0.0
39.47841760435743


In [11]:
K = sp.integrate(
    sp.integrate(
        B0_kappa.T * F * B0_kappa, (x, 0, a)), (y, 0, b)
)

K = np.array(K, dtype=np.longdouble)

In [12]:
print(K.shape)
print(K.min())
print(K.max())

(48, 48)
-494588.65590833756
3777256.11389076


In [13]:
A = K
B = KG

eig_values, eig_vectors = linalg.eig(A, B)

In [14]:
eig_values

array([         inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j, 267.6836608 +0.j,
        71.04685863+0.j,   6.21530421+0.j,  42.79463961+0.j,
        40.16183821+0.j,  17.02487582+0.j,  24.58132626+0.j,
        19.15602047+0.j,  94.45044412+0.j,  98.3894421 +0.j,
        66.2928926 +0.j,   4.14275912+0.j,  37.28608626+0.j,
        10.66518067+0.j,  16.57159885+0.j,  24.86135661+0.j])

## Check with compmech

In [15]:
from compmech.panel.panel import Panel
from compmech.analysis import lb, static

In [16]:
# skin panels
laminaprop = (E1, E2, v12, G12, G12, G12)
stack = [45, 0, 0, 45, 0, 0, 45,]
p1 = Panel(group='skin', Nxx=-1, x0=0, y0=0., a=a, b=b,m=20, n=20, plyt=t, stack=stack, laminaprop=laminaprop)

In [17]:
k0_1 = p1.calc_k0()
kG0_1 = p1.calc_kG0()

eigvals, eigvecs = lb(k0_1, kG0_1, silent=True)
eigvals

		Calculating k0... 
		finished!
		Calculating kG0... 
		finished!


array([ 4.14266552,  6.21517585, 10.66504556, 16.57083361, 17.0241622 ,
       19.15525638, 24.58073993, 24.83828109, 25.26226129, 32.7633788 ,
       35.3000328 , 37.28351341, 40.16085452, 42.6487838 , 42.79306548,
       46.76560192, 47.18712241, 54.41057402, 55.96560942, 60.90740842,
       66.26641915, 67.21581641, 68.18588908, 69.3549638 , 71.03202366])

In [18]:
9.70483681*360

3493.7412516

In [19]:
print(kG0_1.max())

0.0009523809523809536


## Check with composipy

In [20]:
from composipy import Laminate, Ply, buckling_load

In [21]:
ply = Ply(E1, E2, v12, G12, t)
layup_1 = [(45, ply),(0, ply),(0, ply),(45, ply),(0, ply),(0, ply),(45, ply)]
laminate_1 = Laminate(layup_1)

In [22]:
load = buckling_load(a, b, laminate_1.D, shape_plot = False , eig = True, n=10)

In [23]:
a = abs(load['eigen values'])
a.sort()
a

array([4.14270828e+00, 6.21522205e+00, 1.06650914e+01, 1.65710358e+01,
       1.70242153e+01, 1.91554561e+01, 2.45809295e+01, 2.48385030e+01,
       2.52623245e+01, 3.27636136e+01, 3.53001050e+01, 3.72839695e+01,
       4.01613869e+01, 4.26490962e+01, 4.27935413e+01, 4.67661371e+01,
       4.71871495e+01, 5.44108590e+01, 5.59663308e+01, 6.09072064e+01,
       6.62675469e+01, 6.72164889e+01, 6.81862074e+01, 6.93560842e+01,
       7.10330003e+01, 7.64561102e+01, 7.66218673e+01, 8.06194972e+01,
       8.36422449e+01, 8.68293907e+01, 9.38431588e+01, 9.44488356e+01,
       9.59106601e+01, 9.83663621e+01, 9.94734984e+01, 1.00992872e+02,
       1.03585297e+02, 1.06783130e+02, 1.07936385e+02, 1.13012131e+02,
       1.14423916e+02, 1.14554965e+02, 1.25606923e+02, 1.28759733e+02,
       1.31058289e+02, 1.39329386e+02, 1.49151952e+02, 1.52444711e+02,
       1.53256367e+02, 1.55402551e+02, 1.60648911e+02, 1.71172156e+02,
       1.72395090e+02, 1.87014978e+02, 2.01935205e+02, 2.03008923e+02,
      